<a href="https://colab.research.google.com/github/monika-chivate/exam/blob/main/sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

In [ ]:
Tokenizer?

In [ ]:
df = pd.read_csv('/content/sentiment analysis.txt',sep = '\t',names=['text', 'sentiment'])

In [ ]:
df.head(5)

,text,sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       1000 non-null   object
 1   sentiment  1000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 15.8+ KB


In [ ]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(df['text'].values)
X = tokenizer.texts_to_sequences(df['text'].values)
X = pad_sequences(X)

In [ ]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 32, 128)           256000    
                                                                 
 spatial_dropout1d_2 (Spatia  (None, 32, 128)          0         
 lDropout1D)                                                     
                                                                 
 lstm_2 (LSTM)               (None, 196)               254800    
                                                                 
 dense_2 (Dense)             (None, 2)                 394       
                                                                 
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
Y = pd.get_dummies(df['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(670, 32) (670, 2)
(330, 32) (330, 2)


In [ ]:
Y

array([[0, 1],
       [1, 0],
       [1, 0],
       ...,
       [1, 0],
       [1, 0],
       [1, 0]], dtype=uint8)

In [ ]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 2)

Epoch 1/7
21/21 - 6s - loss: 0.6896 - accuracy: 0.5522 - 6s/epoch - 287ms/step
Epoch 2/7
21/21 - 3s - loss: 0.6349 - accuracy: 0.7164 - 3s/epoch - 151ms/step
Epoch 3/7
21/21 - 3s - loss: 0.4683 - accuracy: 0.8149 - 3s/epoch - 148ms/step
Epoch 4/7
21/21 - 3s - loss: 0.2647 - accuracy: 0.9119 - 3s/epoch - 151ms/step
Epoch 5/7
21/21 - 3s - loss: 0.1617 - accuracy: 0.9507 - 3s/epoch - 150ms/step
Epoch 6/7
21/21 - 3s - loss: 0.1095 - accuracy: 0.9687 - 3s/epoch - 146ms/step
Epoch 7/7
21/21 - 3s - loss: 0.0588 - accuracy: 0.9881 - 3s/epoch - 146ms/step


In [ ]:
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

11/11 - 1s - loss: 0.8255 - accuracy: 0.7515 - 613ms/epoch - 56ms/step
score: 0.83
acc: 0.75
